In [1]:
import sys
sys.path.append('/projects/sopro_nmt/badr/conda3/lib/python3.6/site-packages/')

import spacy 
import pandas as pd
from collections import defaultdict

import glob

In [3]:
model = "en_core_web_sm"

In [5]:
text_files = glob.glob("COCA_FICTION/*.txt")

In [6]:
text_files

['COCA_FICTION/w_fic_2004.txt',
 'COCA_FICTION/w_fic_2000.txt',
 'COCA_FICTION/w_fic_1997.txt',
 'COCA_FICTION/w_fic_2009.txt',
 'COCA_FICTION/w_fic_1995.txt',
 'COCA_FICTION/w_fic_2012.txt',
 'COCA_FICTION/w_fic_2011.txt',
 'COCA_FICTION/w_fic_2007.txt',
 'COCA_FICTION/w_fic_2010.txt',
 'COCA_FICTION/w_fic_1996.txt',
 'COCA_FICTION/w_fic_2006.txt',
 'COCA_FICTION/w_fic_2001.txt',
 'COCA_FICTION/w_fic_1992.txt',
 'COCA_FICTION/w_fic_2002.txt',
 'COCA_FICTION/w_fic_2005.txt',
 'COCA_FICTION/w_fic_1998.txt',
 'COCA_FICTION/w_fic_1993.txt',
 'COCA_FICTION/w_fic_2008.txt',
 'COCA_FICTION/w_fic_1994.txt',
 'COCA_FICTION/w_fic_1999.txt',
 'COCA_FICTION/w_fic_2003.txt',
 'COCA_FICTION/w_fic_1990.txt',
 'COCA_FICTION/w_fic_1991.txt']

In [10]:
nlp = spacy.load(model)

In [13]:
relativizer_words = {'who', 'which', 'whose', 'whom', 'which', 'that'}

In [14]:
STEP = 10**5

In [24]:
sent_idx = 0

for _file in text_files:
    
    _name = _file.split('/')[1][:-4]
    
    print('Prcoessing: ', _name)
    
    with open(_file) as f:
        text = f.read()
    
    RC_collection = defaultdict(lambda: defaultdict())
    

    for seg in range(0, len(text), STEP):
        
        # use SpaCy module
        doc = nlp(text[seg:seg+STEP])
        
        for (k, sent) in enumerate(doc.sents):

            words = [w.text for w in sent]

            relativizer_count = len([w for w in words if w in relativizer_words])

            # sometimes the corpus is noisy contains tokens @ @ @ @ @ @
            if words.count('@') > 2: continue 

            if relativizer_count != 1: continue

            # process sentence with RC token by token
            for i in range(len(sent)):

                # to avoid unknow errors 
                try:
                    # check if the relativizer word corresponds to a RC
                    if sent[i].dep_ == "relcl":

                        # then the previous word token should be a relativizer for this to RC
                        if sent[i-1].text in relativizer_words:

                            # THIS IS A RC!
                            RC_words = [w.text for w in sent[i].subtree]

                            RC_collection[sent_idx]['RC'] = ' '.join(RC_words)
                            RC_collection[sent_idx]['RC_subj'] = \
                                [w.text for w in sent[i].subtree if w.dep_ == 'nsubj'][-1]
                            RC_collection[sent_idx]['RC_subj_pos'] = \
                                [w.pos_ for w in sent[i].subtree if w.dep_ == 'nsubj'][-1]
                            RC_collection[sent_idx]['relativizer_word'] = sent[i-1].text
                            RC_collection[sent_idx]['relativizer_func'] = sent[i-1].dep_
                            RC_collection[sent_idx]['RC_modifier_head'] = sent[i].head.text
                            RC_collection[sent_idx]['RC_modifier_NP'] = ' '.join(w for w in \
                                [t.text for t in sent[i].head.lefts] + [sent[i].head.text])
                            RC_collection[sent_idx]['RC_location'] = (i-1, (i-1) + len(RC_words) - 1)
                            RC_collection[sent_idx]['RC_context']  = (sent[i-2].text, \
                                sent[i-2 + len(RC_words) + 1].text) 
                            RC_collection[sent_idx]['sentence'] = sent.text
                            
                            sent_idx += 1

                        # sometimes the distance between relativizer and verb in RC > 1
                        elif sent[i-2].text in relativizer_words:
                            
                            # THIS IS A RC!
                            RC_words = [w.text for w in sent[i].subtree]

                            RC_collection[sent_idx]['RC'] = ' '.join(RC_words)
                            RC_collection[sent_idx]['RC_subj'] = \
                                [w.text for w in sent[i].subtree if w.dep_ == 'nsubj'][-1]
                            RC_collection[sent_idx]['RC_subj_pos'] = \
                                [w.pos_ for w in sent[i].subtree if w.dep_ == 'nsubj'][-1]
                            RC_collection[sent_idx]['relativizer_word'] = sent[i-2].text
                            RC_collection[sent_idx]['relativizer_func'] = sent[i-2].dep_
                            RC_collection[sent_idx]['RC_modifier_head'] = sent[i].head.text
                            RC_collection[sent_idx]['RC_modifier_NP'] = ' '.join(w for w in \
                                [t.text for t in sent[i].head.lefts] + [sent[i].head.text])

                            RC_collection[sent_idx]['RC_location'] = (i-1, (i-1) + len(RC_words) - 1)
                            RC_collection[sent_idx]['RC_context']  = (sent[i-3].text, \
                                sent[i-2 + len(RC_words) ].text) 
                            RC_collection[sent_idx]['sentence'] = sent.text
                            
                            sent_idx += 1
                
                except:
                    pass
        
    print('Number of collected sentences:', len(RC_collection))

    df = pd.DataFrame.from_dict(RC_collection, orient='index')
    df.to_csv('RC_FICTION/' + _name + '.RC.tsv', sep='\t')


w_fic_2004
Number of collected sentence: 8662
w_fic_2000
Number of collected sentence: 8375
w_fic_1997
Number of collected sentence: 7282
w_fic_2009
Number of collected sentence: 9436
w_fic_1995
Number of collected sentence: 7413
w_fic_2012
Number of collected sentence: 5192
w_fic_2011
Number of collected sentence: 9348
w_fic_2007
Number of collected sentence: 8412
w_fic_2010
Number of collected sentence: 8859
w_fic_1996
Number of collected sentence: 7304
w_fic_2006
Number of collected sentence: 8908
w_fic_2001
Number of collected sentence: 8209
w_fic_1992
Number of collected sentence: 8131
w_fic_2002
Number of collected sentence: 7768
w_fic_2005
Number of collected sentence: 8906
w_fic_1998
Number of collected sentence: 7756
w_fic_1993
Number of collected sentence: 7562
w_fic_2008
Number of collected sentence: 9383
w_fic_1994
Number of collected sentence: 7898
w_fic_1999
Number of collected sentence: 8482
w_fic_2003
Number of collected sentence: 8584
w_fic_1990
Number of collected sen

In [16]:
len(RC_collection)

0